<hr style="height:2px;border:none"/>
<img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">

<center>
    <h1> INF495 - Modelamiento Computacional Aplicado </h1>
    <h1> Project</h1>
    <h1> Modeling Fungi</h1>
    <h3>Jorge Sanhueza jorge.sanhuezac@sansano.usm.cl</h3>
</center>
<br>
<hr style="height:2px;border:none"/>

# Context

The project is based on the problem A of The Mathematical Contest in Modeling (MCM) 2021 
https://www.mathmodels.org/Problems/2021/MCM-A/2021_MCM_Problem_A.pdf>




The carbon cycle describes the process of the exchange of carbon throughout the geochemical
cycle of the Earth, and is a vital component for life on the planet.
Part of the carbon cycle includes the decomposition of compounds, allowing carbon to be renewed and used in other forms.
One key
component of this part of the process is the decomposition of plant material and woody fibers.

**Fungi** acts as a key agent in the decomposition of **wooden fibers and ground litter**

Different **fungi traits** affect in the **decomposition rate** of this fibers and ground litter.

For this MCM Problem you should focus on just two traits of a fungus:
 - The **growth rate** of the fungus
 - The fungus **tolerance to moisture**.
 
In particular, the slow growing strains of fungi tend to be better able to survive and grow in the presence of environmental changes with respect to moisture and temperature, while the faster growing strains tend to be less robust to the same changes.

The goal is to model the decomposition of woody fibers in a given patch of land, and do so in the presence of multiple types of fungi breaking down woody fibers in the same area.

As you explore the relationship of the two traits of interest, growth rate and moisture tolerance,
with the rate of decomposition, several questions may arise to include:

Using these two traits
- How do the different fungi **interact** and decompose ground litter in a fixed patch of land in different environments?
- Within these different environments, how will the **decomposition be impacted overtime** as conditions vary?
- How do environmental changes and the variation in environmental change impact the **long-term dynamics** with respect to decomposition, as well as competition between fungi in a given environment?

The estimation for the decomposition rates, given the growth rate, is shown
in Figure 1. The estimation of the decomposition rates, given the relative moisture tolerance, is
shown in Figure 2.


<img src="./figures/figures.png" style="width:70%">

# Agent-based Model 

# Enviroment

Patch of land of size nxm that holds Fungi and fibers. The humidity of the patch are separated by sections of different constant humidity.  

# Agents

Two types of fungi, slow growing strains of fungi (**S-Fungi**) and faster growing strains of fungi (**F-Fungi**).

Wood fibers and dead plant material (**Fibers**).

## S-Fungi
    - Decompose fibers slower
    - Withstands high humidity section
    - Grows slower than F-Fungi
    - When there is no fibers the fungi dies (lenght decreases?)
## F-Fungi
    - Decompose fibers faster
    - Cant Withstand high humidity section
    - Grows faster than S-Fungi
    - When there is no fibers the fungi dies (lenght decreases?)



In [2]:
import numpy as np

In [11]:
n = 10


In [47]:

terreno = np.zeros((n,n))

n_Sfungi = 1
n_Ffungi = 0
Sfungi = []
Ffungi = []

def Fungi:




array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [44]:
np.random.randint(n)

4

1.5